In [1]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 15:11:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
# spark.sql(f"DROP NAMESPACE IF EXISTS {test_namespace} CASCADE")
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
TARGET_CATEGORY = 'health_beauty'

In [4]:
sale_stats = spark.read.table('warehousedev.gold.sale_stats')
sale_stats.show(n=5)

+--------------------+---------------+-----------+-----------+---------+
|          product_id|       category|order_count|total_sales|mean_sale|
+--------------------+---------------+-----------+-----------+---------+
|aca2eb7d00ea1a7b8...|furniture_decor|        520|   44197.75|  84.9957|
|422879e10f4668299...|   garden_tools|        484|   34201.26|  70.6638|
|99a4788cb24856965...| bed_bath_table|        477|    49907.5| 104.6279|
|389d119b48cf3043d...|   garden_tools|        390|   28543.65|  73.1888|
|368c6c730842d7801...|   garden_tools|        388|    27984.4|  72.1247|
+--------------------+---------------+-----------+-----------+---------+
only showing top 5 rows



In [9]:
target_product_sales_stats = sale_stats.filter(F.col('category') == TARGET_CATEGORY)

In [11]:
# 1. 기준점(Threshold) 계산
# percentile_approx 함수를 사용하여 분위수 계산
order_count_threshold = target_product_sales_stats.agg(
    F.expr("percentile_approx(order_count, 0.75)")
).collect()[0][0]

median_avg_price = target_product_sales_stats.agg(
    F.expr("percentile_approx(mean_sale, 0.5)")
).collect()[0][0]

# print(order_count_threshold)
# print(median_avg_price)


# 2. 'group' 컬럼 추가
# when/otherwise와 col 함수를 사용하여 조건에 따라 그룹을 분류합니다.
health_beauty_sales_stats_bcg = target_product_sales_stats.withColumn("segment",
    F.when((F.col("order_count") >= order_count_threshold) & (F.col("mean_sale") >= median_avg_price), "Star Products")
    .when((F.col("order_count") >= order_count_threshold) & (F.col("mean_sale") < median_avg_price), "Volume Drivers")
    .when((F.col("order_count") < order_count_threshold) & (F.col("mean_sale") >= median_avg_price), "Niche Gems")
    .otherwise("Question Marks")
)

health_beauty_sales_stats_bcg.writeTo(f"{test_namespace}.health_beauty_sales_stats_bcg").createOrReplace()

In [13]:
spark.stop()

# 구매기록에 Segment join
- Segment type: `Start Product`, `Volume Drivers`, `Niche Gems`, `Question Marks`

In [8]:
complete_target_category_order_df.show(n=4)

+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+
|            order_id|          product_id|     category|order_item_id|           seller_id|total_price|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|
+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+
|0005a1a1728c9d785...|310ae3c140ff94b03...|health_beauty|            1|a416b6a846a117243...|      157.6|639d23421f5517f69...|2018-03-19 18:40:33|2018-03-20 18:35:21|2018-03-28 00:37:42|2018-03-29 18:17:31|
|00061f2a7bc09da83...|d63c1011f49d98b97...|health_beauty|            1|cc419e0650a3c5ba7...|      68.87|107e6259485efac66...|2018-03-24 22:16:10|2018-03-24 22:28:09|2018-03-27 

In [9]:
target_product_order_info_with_segment = complete_target_category_order_df.join(target_product_sales_stats_segment.select('product_id', 'segment'), on='product_id', how='inner')
target_product_order_info_with_segment.show(n=5)

+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------+
|          product_id|            order_id|     category|order_item_id|           seller_id|total_price|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|      segment|
+--------------------+--------------------+-------------+-------------+--------------------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------+
|05f0fe07929d35be0...|362cd36939c1c4638...|health_beauty|            1|c70c1b0d8ca86052f...|     118.35|32ca12567c820aa0b...|2018-06-25 23:00:42|2018-06-27 08:53:50|2018-06-29 14:16:00|2018-07-02 21:05:36|Star Products|
|05f0fe07929d35be0...|b41cc5a0872a7fd88...|health_beauty|            1|c70c1b0d8ca86052f...|     118.35|ada8dea5e28ec3e9

In [10]:
target_product_order_info_with_segment.writeTo(f"{test_namespace}.target_product_order_info_with_segment").createOrReplace()

In [11]:
spark.stop()